# Running on hardware
This notebook demonstrates how to run on actual hardware. To run it you require an IBM Quantum Experience Account which can be obtained here

https://quantum-computing.ibm.com/login

After obtaining the IBM Quantum Experience Account follow the instructions here

https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq

to save your API token. Afterwards you should be able to access the publically available IBM Quantum Systems.

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.visualization import *
from qiskit.quantum_info import state_fidelity
# Magic function to render plots in the notebook after the cell executing the plot command
%matplotlib inline

## For demonstration purposes we prepare a little quantum circuit

In [ ]:
qc = QuantumCircuit(3)
qc.h(0)
qc.cnot(0,1)
qc.cnot(1,2)
qc.measure_active()
qc.draw(output='mpl')

## Before we can access the quantum systems we have the authenticate against IBM Quantum Experience with our stored credentials

In [ ]:
# Loading your IBM Q account
IBMQ.load_account()
provider = IBMQ.get_provider(group='open')

## Select a suitable device and run on the hardware

In [ ]:
# List the existing quantum devices and their number of qubits
device_list = provider.backends(simulator=False)
for dev in device_list:
    print(dev.name() + ': ' + str(dev.configuration().n_qubits) + ' qubits')

In [ ]:
# We choose one device which has enough qubits for our experiment and send the job to the device
num_shots_hardware = 1024
hardware_backend = provider.get_backend('ibmq_quito')
job = execute(qc, backend=hardware_backend, shots=num_shots_hardware)

## After running successfully we can retrieve the results and visualize them

In [ ]:
result = job.result()
counts = result.get_counts()
plot_histogram(counts, title='Run on hardware')

## For comparison we can also run on the ideal simulator with a varying number of shots

In [ ]:
simulator_backend = Aer.get_backend('qasm_simulator')
counts_simulator = list()
num_shots_simulator = [1024, 8192]
for num_shots in num_shots_simulator:
    job = execute(qc, backend=simulator_backend, shots=num_shots)
    result_simulator = job.result()
    counts_simulator.append(result_simulator.get_counts())

In [ ]:
# We plot a comparison between the different results
plot_histogram([counts] + counts_simulator, title='Hardware vs. simulator',
               legend=['hardware ' + str(num_shots_hardware) + ' shots',
                       'simulator ' + str(num_shots_simulator[0]) + ' shots',
                       'simulator ' + str(num_shots_simulator[1]) + ' shots'])